In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os, glob, time
from collections import OrderedDict
import numpy as np
import arviz as az
import enterprise
from enterprise.pulsar import Pulsar

import warnings
warnings.filterwarnings('ignore')

In [3]:
from enterprise.signals import utils
import ephem

In [15]:
current_path = os.getcwd()
splt_path = current_path.split("/")
#top_path_idx = splt_path.index("akaiser")
top_path_idx = splt_path.index("nanograv")
top_dir = "/".join(splt_path[0 : top_path_idx + 1])
def get_psr(psrlist,datarelease):
    
    datadir = top_dir + '/{}'.format(datarelease)

    parfiles = sorted(glob.glob(datadir + "/par/*.par"))
    timfiles = sorted(glob.glob(datadir + "/tim/*.tim"))

    # filter
    parfiles = [
        x for x in parfiles if x.split("/")[-1].split(".")[0].split("_")[0] in psrlist
    ]
    timfiles = [
        x for x in timfiles if x.split("/")[-1].split(".")[0].split("_")[0] in psrlist
    ]

    psr = Pulsar(parfiles[0], timfiles[0], ephem='DE436', clk=None, drop_t2pulsar=False)
                        
    if 'RAJ' in psr.t2pulsar.pars():
        print('Original Equatorial')
        filename = psr.t2pulsar.parfile.split('/')[-1]
        file = glob.glob('../../*/*/'+filename)[0]
        with open(file,'r') as f:
            for line in f.readlines():
                if line.split()[0] in ['RA','RAJ']:
                    val_ra = line.split()[1]
                    error_ra = '0:0:' + line.split()[-1]
                elif line.split()[0] in ['DEC','DECJ']:
                    val_dec = line.split()[1]
                    error_dec = '0:0:' + line.split()[-1]
                    
        eq_from_file = ephem.Equatorial(val_ra,val_dec)
        ec_from_file = ephem.Ecliptic(eq_from_file)
        
        eq_err_from_file = ephem.Equatorial(error_ra,error_dec)
        ec_err_from_file = ephem.Ecliptic(eq_err_from_file)
        
        eq_from_enterprise = ephem.Equatorial(np.double(psr.t2pulsar['RAJ'].val),np.double(psr.t2pulsar['DECJ'].val))
        ec_from_enterprise = ephem.Ecliptic(eq_from_enterprise)
        
        eq_err_from_enterprise = ephem.Equatorial(np.double(psr.t2pulsar['RAJ'].err),np.double(psr.t2pulsar['DECJ'].err))
        ec_err_from_enterprise = ephem.Ecliptic(eq_err_from_enterprise)
        
    else:
        print('Original Ecliptic')
        filename = psr.t2pulsar.parfile.split('/')[-1]
        file = glob.glob('../../*/*/'+filename)[0]
        with open(file,'r') as f:
            for line in f.readlines():
                if line.split()[0] in ['ELONG','LAMBDA']:
                    val_elong = ephem.degrees(line.split()[1])
                    error_elong = ephem.degrees(line.split()[-1])
                elif line.split()[0] in ['ELAT','BETA']:
                    val_elat = ephem.degrees(line.split()[1])
                    error_elat = ephem.degrees(line.split()[-1])
        
        
        ec_from_file = ephem.Ecliptic(val_elong,val_elat)
        eq_from_file = ephem.Equatorial(ec_from_file)
        
        ec_err_from_file = ephem.Ecliptic(error_elong,error_elat)
        eq_err_from_file = ephem.Equatorial(ec_err_from_file)
        
        ec_from_enterprise = ephem.Ecliptic(psr.t2pulsar['ELONG'].val,psr.t2pulsar['ELAT'].val)
        eq_from_enterprise = ephem.Equatorial(ec_from_enterprise)
        
        ec_err_from_enterprise = ephem.Ecliptic(psr.t2pulsar['ELONG'].err,psr.t2pulsar['ELAT'].err)
        eq_err_from_enterprise = ephem.Equatorial(ec_err_from_enterprise)
        
    return [ec_from_file,eq_from_file,ec_err_from_file,eq_err_from_file,
            ec_from_enterprise,eq_from_enterprise,ec_err_from_enterprise,eq_err_from_enterprise]

In [16]:
psrlist = ["J1640+2224"] #J1643-1224 J1640+2224 J1909-3744 J0030+0451
datareleases = ['5yr','9yr']#,'11yr']#,'12p5yr']

In [17]:
for datarelease in datareleases:
    print('Dataset: ',datarelease)
    [ec_from_file,eq_from_file,ec_err_from_file,eq_err_from_file,
     ec_from_enterprise,eq_from_enterprise,ec_err_from_enterprise,eq_err_from_enterprise] = get_psr(psrlist,
                                                                                                    datarelease)
    print('Ecliptic long:')
    print('From Par File: ',np.double(ec_from_file.lon))
    print('From enterprise: ',np.double(ec_from_enterprise.lon))
    print('Error From Par File: ',np.double(ec_err_from_file.lon))
    print('Error From enterprise: ',np.double(ec_err_from_enterprise.lon))
    print('Ecliptic lat:')
    print('From Par File: ',np.double(ec_from_file.lat))
    print('From enterprise: ',np.double(ec_from_enterprise.lat))
    print('Error From Par File: ',np.double(ec_err_from_file.lat))
    print('Error From enterprise: ',np.double(ec_err_from_enterprise.lat))
    print('RA:')
    print('From Par File: ',np.double(eq_from_file.ra))
    print('From enterprise: ',np.double(eq_from_enterprise.ra))
    print('Error From Par File: ',np.double(eq_err_from_file.ra))
    print('Error From enterprise: ',np.double(eq_err_from_enterprise.ra))
    print('DEC:')
    print('From Par File: ',np.double(eq_from_file.dec))
    print('From enterprise: ',np.double(eq_from_enterprise.dec))
    print('Error From Par File: ',np.double(eq_err_from_file.dec))
    print('Error From enterprise: ',np.double(eq_err_from_enterprise.dec))
    print('\n')

Dataset:  5yr
Original Equatorial
Ecliptic long:
From Par File:  4.2584125964886415
From enterprise:  4.2584125964886415
Error From Par File:  4.7936615215460435e-09
Error From enterprise:  4.793661521546043e-09
Ecliptic lat:
From Par File:  0.7689668876421745
From enterprise:  0.7689668876421746
Error From Par File:  1.143663397803717e-09
Error From enterprise:  1.1436633978037176e-09
RA:
From Par File:  4.364540751695362
From enterprise:  4.364540751695362
Error From Par File:  3.9431752845012275e-09
Error From enterprise:  3.943175284501227e-09
DEC:
From Par File:  0.3909971108371538
From enterprise:  0.3909971108371539
Error From Par File:  2.9560996983061546e-09
Error From enterprise:  2.956099698306155e-09


Dataset:  9yr
Original Ecliptic
Ecliptic long:
From Par File:  4.258412933725018
From enterprise:  4.258412933725018
Error From Par File:  2.570712908554968e-10
Error From enterprise:  nan
Ecliptic lat:
From Par File:  0.768966238239606
From enterprise:  0.7689662382396061
Er

In [7]:
elong = ephem.degrees('243.9890903977685')
elong_err = ephem.degrees('0.0000000147291')
elat = ephem.degrees('44.0585168795088')
elat_err = ephem.degrees('0.0000000179060')
"""elong =  '284:13:15.2'
elat = '44:03:30.8'"""
ra = '16:40:16.7435004'
ra_err = ephem.degrees('0.00005422255240373231')
dec = '22:24:08.94331'
dec_err = ephem.degrees('0.00060973933151821908')

eq_from_eq = ephem.Equatorial(ra,dec)
ec_from_eq = ephem.Ecliptic(eq_from_eq)

eq_err_from_eq = ephem.Equatorial(ra_err,dec_err)
ec_err_from_eq = ephem.Ecliptic(eq_err_from_eq)

ec_from_ec = ephem.Ecliptic(elong, elat)
eq_from_ec = ephem.Equatorial(ec_from_ec)

ec_err_from_ec = ephem.Ecliptic(elong_err, elat_err)
eq_err_from_ec = ephem.Equatorial(ec_err_from_ec)

print('Ecliptic long:')
print('By Hand from Eq: ',np.double(ec_from_eq.lon))
print('By Hand from Ec: ',np.double(ec_from_ec.lon))
print('Error By Hand from Eq: ',np.double(ec_err_from_eq.lon))
print('Error By Hand from Ec: ',np.double(ec_err_from_ec.lon))
print('Ecliptic lat:')
print('By Hand from Eq: ',np.double(ec_from_eq.lat))
print('By Hand from Ec: ',np.double(ec_from_ec.lat))
print('Error By Hand from Eq: ',np.double(ec_err_from_eq.lat))
print('Error By Hand from Ec: ',np.double(ec_err_from_ec.lat))
print('RA:')
print('By Hand from Eq: ',np.double(eq_from_eq.ra))
print('By Hand from Ec: ',np.double(eq_from_ec.ra))
print('Error By Hand from Eq: ',np.double(eq_err_from_eq.ra))
print('Error By Hand from Ec: ',np.double(eq_err_from_ec.ra))
print('DEC:')
print('By Hand from Eq: ',np.double(eq_from_eq.dec))
print('By Hand from Ec: ',np.double(eq_from_ec.dec))
print('Error By Hand from Eq: ',np.double(eq_err_from_eq.dec))
print('Error By Hand from Ec: ',np.double(eq_err_from_ec.dec))
print('\n')

Ecliptic long:
By Hand from Eq:  4.2584125964886415
By Hand from Ec:  4.258412966387142
Error By Hand from Eq:  5.101398370483393e-06
Error By Hand from Ec:  2.570712908554968e-10
Ecliptic lat:
By Hand from Eq:  0.7689668876421745
By Hand from Ec:  0.7689661830929264
Error By Hand from Eq:  9.387365197744324e-06
Error By Hand from Ec:  3.1251865586210467e-10
RA:
By Hand from Eq:  4.364540751695362
By Hand from Ec:  4.364540890262736
Error By Hand from Eq:  9.463620682802945e-07
Error By Hand from Ec:  1.1154551600382968e-10
DEC:
By Hand from Eq:  0.3909971108371538
By Hand from Ec:  0.3909963687873827
Error By Hand from Eq:  1.0641958913902157e-05
Error By Hand from Ec:  3.8898734774206654e-10




In [8]:
tmp = 0.0000000147291
tmp1 = 0.0000000179060
tmp2 = '0:0:0.0000000147291'
tmp3 = '0:0:0.0000000179060'
sadf = ephem.Ecliptic(tmp, tmp1)
sadfs = ephem.Ecliptic(tmp2, tmp3)
print(np.double(sadf.lon),np.double(sadf.lat))
print(np.double(sadfs.lon),np.double(sadfs.lat))

1.47291e-08 1.7906e-08
7.140869190430466e-14 8.681073773947352e-14


In [9]:
from astropy import units as u
from astropy.coordinates import SkyCoord
c = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')